# Quantitative Portfolio Management with Random Forest

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import yfinance as yf
import scipy.optimize as sco
plt.style.use('seaborn-v0_8')

In [ ]:
tickers = ['AAPL', 'MSFT', 'META', 'INTC', 'DIS', 'AMZN', 'KO', 'TSLA', 'BTC-USD', 'GLD']
tickers.sort()
start = "2015-01-01"
end = "2024-12-31"
stocks = pd.DataFrame()
for stock in tickers:
    stocks[stock] = yf.download(stock, start, end)['Close']

In [ ]:
stocks = stocks.dropna()
daily_ret = np.log(stocks / stocks.shift(1)).dropna()
annual_ret = daily_ret.mean() * 252
cov_mat = daily_ret.cov() * 252

In [ ]:
features = []
target = []
lookback = 10
for i in range(lookback, len(daily_ret)):
    features.append(daily_ret.iloc[i-lookback:i].values.flatten())
    target.append(daily_ret.iloc[i].values)
features = np.array(features)
target = np.array(target)
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
predicted_daily_return = model.predict(features[-1].reshape(1, -1))[0]
predicted_annual_return = predicted_daily_return * 252

In [ ]:
def portfolio_volatility(weights, returns, cov_matrix):
    return np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
def negative_sharpe(weights, returns, cov_matrix, rf=0):
    port_return = np.dot(weights, returns)
    port_vol = portfolio_volatility(weights, returns, cov_matrix)
    return -(port_return - rf) / port_vol
n_assets = len(tickers)
initial_weights = np.ones(n_assets) / n_assets
bounds = tuple((0, 1) for _ in range(n_assets))
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
opt = sco.minimize(negative_sharpe, initial_weights, args=(predicted_annual_return, cov_mat),
                   method='SLSQP', bounds=bounds, constraints=constraints)
opt_weights = opt.x

In [ ]:
portfolio = pd.DataFrame({'Asset': tickers, 'Weight': opt_weights})
print(portfolio.sort_values(by='Weight', ascending=False))

In [ ]:
plt.figure(figsize=(12,6))
plt.bar(portfolio['Asset'], portfolio['Weight'])
plt.title('Optimal Portfolio Allocation (ML-based Expected Returns)')
plt.xlabel('Asset')
plt.ylabel('Weight')
plt.grid(True)
plt.show()